In [1]:
! pip install --upgrade python-lifelib notebook imageio matplotlib

  Using cached python_lifelib-2.5.6-py3-none-any.whl
  Using cached matplotlib-3.10.7-cp313-cp313-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached numpy-2.3.4-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached notebook_shim-0.2.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached jupyter_events-0.12.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached jupyter_server_terminals-0.5.3-py3-none-any.whl.metadata (5.6 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached Send2Trash-1.8.3-py3-none-any.whl.metadata (4.0 kB)
  Using cached terminado-0.18.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached babel-2.17.0-py3-none-any.whl.metadat

In [2]:
import lifelib
import os
lt = lifelib.load_rules('Ising_temp.rule').lifetree()

/home/aarondominic/cellular-automata/ising/ising/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generating code for rules ['xising_temp']...
Compiling lifelib shared object (estimated time: 20.0 seconds)...


Estimated compilation progress: 100%|██████████| 100/100 [00:43<00:00,  2.32it/s]

...compilation complete!


In [17]:
W = 100
H_i = 50
H_g = 30

boundary = lt.pattern()
for x in range(-1,W+1):
    for y in range(1,H_g+2):
        if y==H_g+1 or x==-1 or x==W:
            boundary[x,y] = 16
for x in range(W):
    boundary[x,0] = 34 + (1+x)%2

ac = lt.pattern()
ac[0,0]=33

ising_t = lt.load('chess.mc')(0,1).replace(ac, lt.pattern()) # type: ignore



def genpatt():

    rand50 = lt.pattern()
    rand50[0:W, -H_i:0] = { 32: 0.4, 0: 0.4 }

    ising = ising_t & rand50

    gas = lt.pattern()
    rhom={}
    for i in range(16):
        rhom[i] = 1/16
    #gas[0:W, 1:H_g+1] = rhom

    pat = ising|gas|boundary
    return pat

In [19]:
import numpy as np

data=[]

for it in range(50):
    pat = genpatt()
    pat.save(f'h{it}test.rle')
    eq = pat[50000] # for 50x50, 10000 is fine. 50x100 20000.

    for i in range(50):
        rho = np.sum(np.bitwise_count(eq[eq[0:W, 1:H_g+1].coords()]))/((W*H_g)/2*4) # type: ignore
        m = (eq[0:W, -H_i-1:-1].population/(W*H_i))*2 -1

        data.append((float(rho),m))

        eq[0:W, H_g:H_g+1] = 0
        eq = eq[2000]
    eq.save(f'd{it}test.rle')

    print(f'it {it} done!')

with open('../data/cadata.txt','w') as fh:
    fh.write(str(data))

it 0 done!
it 1 done!
it 2 done!
it 3 done!
it 4 done!
it 5 done!
it 6 done!
it 7 done!
it 8 done!
it 9 done!
it 10 done!
it 11 done!
it 12 done!
it 13 done!
it 14 done!
it 15 done!
it 16 done!
it 17 done!
it 18 done!
it 19 done!
it 20 done!
it 21 done!
it 22 done!
it 23 done!
it 24 done!
it 25 done!
it 26 done!
it 27 done!
it 28 done!
it 29 done!
it 30 done!
it 31 done!
it 32 done!
it 33 done!
it 34 done!
it 35 done!
it 36 done!
it 37 done!
it 38 done!
it 39 done!
it 40 done!
it 41 done!
it 42 done!
it 43 done!
it 44 done!
it 45 done!
it 46 done!
it 47 done!
it 48 done!
it 49 done!


In [18]:
chess = lt.pattern()
for x in range(W):
    for y in range(-1,-H_i-1,-1):
            chess[x,y] = 32 + (x+y)%2
chess.save('chess.mc')